In [1]:
import pandas as pd 
import numpy as np 

In [5]:
train = np.load('train_preprocessed.npy', allow_pickle=True )

In [6]:
train.shape

(550784,)

In [15]:
train[0]

{'given': array([[9.5041323e-01, 4.8211467e-01, 9.2348807e-02, 9.2435157e-01,
         8.7563485e-01, 0.0000000e+00, 3.4531930e-03, 7.4778247e-01,
         0.0000000e+00, 3.0276409e-04, 1.0000000e+00, 9.8877323e-01,
         4.7489539e-01, 5.1176184e-01, 4.1326246e-01, 6.0996467e-01,
         7.4185000e-04, 3.7985889e-03, 8.6159718e-01, 8.6855656e-01,
         5.7556349e-01, 5.9469765e-01, 2.6509967e-01, 1.6594577e-01,
         1.7335278e-01, 0.0000000e+00, 1.1757903e-01, 2.7688432e-01,
         1.6001400e-02, 5.9639150e-01, 4.0206063e-01, 4.1554055e-01,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.0000000e+00,
         7.5000000e-01, 0.0000000e+00, 9.1134632e-01, 8.4772384e-01,
         6.7977655e-01, 4.1184431e-01, 4.3498287e-01, 8.4418401e-02,
         7.0660597e-01, 0.0000000e+00, 0.0000000e+00, 3.0887550e-01,
         5.1020408e-01, 7.7428329e-01, 6.9515103e-01, 0.0000000e+00,
         6.1130112e-01, 4.8246446e-01, 5.1273537e-01, 5.1076430e-01,
         9.9067557e-01, 2

In [7]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras

In [49]:
class Convol2anomal(keras.Model) :
    def __init__(self, original_dim) :
        super(self, Convol2anomal).__init__()
        self.original_dim = original_dim
        self.conv2d_1 = layers.Conv2D(filters=32  , kernel_size = (3,3) , padding='same')
        self.maxpool_1 = layers.MaxPool2D()
        self.conv2d_2 = layers.Conv2D(filters=16, kernel_size=(3,3) , padding='same')
        self.maxpool_2 = layers.MaxPooling2D()
        self.flatten = layers.Flatten()
        self.Dense_1 = layers.Dense(63)
    
    def call(self, inputs) : 
        x = self.conv2d_1(inputs)
        x = self.maxpool_1(x)
        x = self.conv2d_2(x)
        x = self.maxpool_2(x)
        x = self.flatten(x)
        return self.Dense_1(x)


In [17]:
given =  np.array([i['given'] for i in train ])
answer = np.array([i['answer'] for i in train ])

In [50]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
mse_loss_fn = tf.keras.losses.MeanSquaredError()

loss_metric = tf.keras.metrics.Mean()

In [51]:
Convol2anomal(original_dim = (56,10)).compile( optimizer = optimizer, loss=mse_loss_fn , metrics=loss_metric)

TypeError: super() argument 1 must be type, not Convol2anomal

In [32]:
train_dataset = tf.data.Dataset.from_tensor_slices(given)
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

In [52]:
train_dataset[-1]

TypeError: 'BatchDataset' object is not subscriptable

In [39]:
for epoch in range(1):
  print('Start of epoch %d' % (epoch,))

  # Iterate over the batches of the dataset.
  for step, x_batch_train in enumerate(train_dataset):
    with tf.GradientTape() as tape:
      # !!! uncomment the following two lines to use workaround and skip !!!
      # if step == 0 and epoch == 0:
      #   vae._set_inputs(x_batch_train)
      reconstructed = Convol2anomal(x_batch_train)
      # Compute reconstruction loss
      loss = mse_loss_fn(x_batch_train, reconstructed)
      loss += sum(vae.losses)  # Add KLD regularization loss

    grads = tape.gradient(loss, Convol2anomal.trainable_weights)
    optimizer.apply_gradients(zip(grads, Convol2anomal.trainable_weights))

    loss_metric(loss)

    if step % 100 == 0:
      print('step %s: mean loss = %s' % (step, loss_metric.result()))

Start of epoch 0


TypeError: __init__() takes 1 positional argument but 2 were given